In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize, differential_evolution
from scipy.interpolate import InterpolatedUnivariateSpline, interp1d, splrep
from scipy.stats import norm
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
import george
from george import kernels

In [ ]:
# Import the .json file

os.chdir(r"C:\Users\ricky\JupyterNotebooks\Intern21\import_photometry_data\selected_all_type_photometry")
filename = glob.glob('*.json')
np.random.seed(1)
np.random.shuffle(filename)
print(len(filename))
print(filename)

# Create a list for all .json, the 1st SN saved as json_data[0], the 2nd SN saved as json_data[1], etc.
json_data = []
for i in filename:
    with open(i, encoding="utf-8") as f:
        json_data.append(json.load(f))

In [ ]:
def avoid_emptySN(filename, Filter):
    
    Band = []
    filename_id = []
    #filename1 = filename
    #print(filename)
    for i in range(len(filename)):
        #print(i)
        Band.append([])
        
        SN_name = filename[i].replace('.json', '')
        SN_name = SN_name.replace('_', ':')
        
        N = len(json_data[i][SN_name]['photometry']) # The no. of data point of photometry in each SN
        #print(SN_name)
        for j in range(N): # Loop through all photemetry datapoint in one SN
            # Avoid any data point without band data
            try:
                Band[i].append(json_data[i][SN_name]['photometry'][j]['band'])
            except:
                Band[i].append(0)
                
        a = 0
        for j in range(N):
            for k in range(len(Filter)):
                if Band[i][j] == Filter[k]:
                    a += 1
        
        if a > 10: # Any SN with more than 10 data points will be used
            filename_id.append(i)
            
    return filename_id

In [ ]:
Filter_JMC = ['U', 'B', 'V', 'R', 'I'] # the Johnson-Morgon-Cousins UBVRI filter system
Filter_SDSS_p = ["u'", "g'", "r'", "i'"] # the SDSS primed filter system
Filter_SDSS = ['u', 'g', 'r', 'i'] # the SDSS filter system
# For now the primed and unprimed SDSS filter will be treated same

In [ ]:
filename_JMC = avoid_emptySN(filename, Filter_JMC)
filename_SDSS_p = avoid_emptySN(filename, Filter_SDSS_p)
filename_SDSS = avoid_emptySN(filename, Filter_SDSS)

print(len(filename_JMC), len(filename_SDSS_p), len(filename_SDSS))

In [ ]:
print(filename[filename_JMC[245]])

In [ ]:
'''Extract data from .json

Input
    Filter: list of string
        1D list
        Name of each band of a particular filter system in use
    
Return
    Time: list 
        shape=(len(Filter), len(filename), number of data)
        Time of datapoints of SN
        
    Magnitude_Abs: list
        shape=(len(Filter), len(filename), number of data)
        Absolute magnitude of datapoints of SN,
        calculated from the relative magnitude, redshift, and luminosity distance
        
    Magnitude_Abs_err: list
        shape=(len(Filter), len(filename), number of data)
        Error of absolute magnitude of datapoints of SN,
        calculated from relative magnitude error and luminosity distance error
        
    Type: list
        1D list
        Store the claimed type of the SN
'''

def lc_extractor(filename, filename_id, Filter):
    
    Band = [] # Contain EM band chosen for analysis
    Type = [] # Claimed type from the .json
    
    Time = [] # Contain time (MJD) for each band
    Magnitude_Abs = [] # Contain absolute magnitude for each band
    Magnitude_Abs_err = [] # Contain error of absolute magnitude for each band
    
    for j in range(len(Filter)): # Create a dimension to store the data at each band separately
        Time.append([]) 
        Magnitude_Abs.append([])
        Magnitude_Abs_err.append([])

    
    
    for i in range(len(filename_id)): # Loop through all SN

        Band.append([]) # Create 2D list

        SN_name = filename[filename_id[i]].replace('.json', '')
        SN_name = SN_name.replace('_', ':')
        
        LumDist = float(json_data[filename_id[i]][SN_name]['lumdist'][0]['value']) # Obtain the luminosity distance
        
        try:
            LumDist_err = float(json_data[filename_id[i]][SN_name]['lumdist'][0]['e_value'])
        except:
            LumDist_err = 0

        z = float(json_data[filename_id[i]][SN_name]['redshift'][0]['value']) # Obtain the redshift, z

        N = len(json_data[filename_id[i]][SN_name]['photometry']) # The no. of data point of photometry in each SN

        Type.append(json_data[filename_id[i]][SN_name]['claimedtype'][0]['value']) 

        for j in range(len(Filter)):

            Time[j].append([])
            Magnitude_Abs[j].append([])
            Magnitude_Abs_err[j].append([])

        for j in range(N): # Loop through all photemetry datapoint in one SN
            # Avoid any data point without band data
            try:
                Band[i].append(json_data[filename_id[i]][SN_name]['photometry'][j]['band'])
            except:
                Band[i].append(0)

            for k in range(len(Filter)):
                # Create light curves for every sources   
                if Band[i][j] == Filter[k]:

                    Magnitude_App = float(json_data[filename_id[i]][SN_name]['photometry'][j]['magnitude']) # Obtain the apparent magnitude from photometry       

                    Time[k][i].append(float(json_data[filename_id[i]][SN_name]['photometry'][j]['time'])) # Fill the Time list
                    Magnitude_Abs[k][i].append(Magnitude_App - 5*np.log10(LumDist*1e5) + 2.5*np.log10(1+z)) # Calculate the absolute magnitude and fill the Magnitude_Abs list

                    try:
                        Magnitude_App_err = float(json_data[filename_id[i]][SN_name]['photometry'][j]['e_magnitude'])
                        Magnitude_Abs_err[k][i].append(np.sqrt(Magnitude_App_err**2 + 5*0.434*LumDist_err/LumDist))
                    except:
                        Magnitude_Abs_err[k][i].append(0.3)
            
            
    return Time, Magnitude_Abs, Magnitude_Abs_err, Type

In [ ]:
Time_JMC, Magnitude_Abs_JMC, Magnitude_Abs_err_JMC, Type_JMC = lc_extractor(filename, filename_JMC, Filter_JMC)
Time_SDSS_p, Magnitude_Abs_SDSS_p, Magnitude_Abs_err_SDSS_p, Type_SDSS_p = lc_extractor(filename, filename_SDSS_p, Filter_SDSS_p)
Time_SDSS, Magnitude_Abs_SDSS, Magnitude_Abs_err_SDSS, Type_SDSS = lc_extractor(filename, filename_SDSS, Filter_SDSS)

print(np.array(Time_JMC).shape)

In [ ]:
# Convert the data lists into shape=(len(filename_id), len(Filter), data points)
def transpose(Time, Magnitude_Abs, Magnitude_Abs_err):
    return np.array(Time, dtype="object").T, np.array(Magnitude_Abs, dtype="object").T, np.array(Magnitude_Abs_err, dtype="object").T

In [ ]:
t_JMC, m_JMC, m_err_JMC = transpose(Time_JMC, Magnitude_Abs_JMC, Magnitude_Abs_err_JMC)
t_SDSS_p, m_SDSS_p, m_err_SDSS_p = transpose(Time_SDSS_p, Magnitude_Abs_SDSS_p, Magnitude_Abs_err_SDSS_p)
t_SDSS, m_SDSS, m_err_SDSS = transpose(Time_SDSS, Magnitude_Abs_SDSS, Magnitude_Abs_err_SDSS)

In [ ]:
'''Zeroing the time list by the peak magnitude such that the peak has day = 0
Also truncate the data lists to only include -50 days to 135 days post peak'''
def peak_zeroing(t, m, m_err, Filter, lightcurve_length_before, lightcurve_length_after):
    
    for i in range(len(t)):
        # Find the maximum time in shortest wavelength band
        maximum = 0
        maximum1 = []
        a = None

        if len(m[i][1]) != 0: # choosing B band for maximum calculation as default
            a = 1 
        else:
            for j in range(len(Filter)): # choose other bands if B band is empty
                if len(m[i][j]) != 0:
                    a = j
                    break

        maximum = np.argmin(m[i][a]) # save the id of maximum magnitude, in B band or otherwise
        t_max = t[i][a][maximum] # save the time of maximum magnitude

        for j in range(len(Filter)):

            t[i][j] = np.array(t[i][j]) - t_max # shift the days of time list such that the peak is 0 days

            t[i][j] = np.delete(t[i][j], np.where(t[i][j] > lightcurve_length_after)) # truncate away any time data after 135 days
            m[i][j] = m[i][j][0:len(t[i][j])] # truncate away any magnitude after 135 days
            m_err[i][j] = m_err[i][j][0:len(t[i][j])] # truncate away any magnitude error after 135 days

            t[i][j] = np.delete(t[i][j], np.where(t[i][j] < lightcurve_length_before)) # truncate away any time data before -50 days
            m[i][j] = m[i][j][len(m[i][j]) - len(t[i][j]):] # truncate away any magnitude data before -50 days
            m_err[i][j] = m_err[i][j][len(m_err[i][j]) - len(t[i][j]):] # truncate away any magnitude error data before -50 days

            if (len(t[i][j]) - len(m_err[i][j])) !=0:
                print('bruh') # print bruh if the length of the lists doesn't match

    return t, m, m_err

In [ ]:
lightcurve_length_before = -50
lightcurve_length_after = 135

t_JMC, m_JMC, m_err_JMC = peak_zeroing(t_JMC, m_JMC, m_err_JMC, Filter_JMC, lightcurve_length_before, lightcurve_length_after)
t_SDSS_p, m_SDSS_p, m_err_SDSS_p = peak_zeroing(t_SDSS_p, m_SDSS_p, m_err_SDSS_p, Filter_SDSS_p, lightcurve_length_before, lightcurve_length_after)
t_SDSS, m_SDSS, m_err_SDSS = peak_zeroing(t_SDSS, m_SDSS, m_err_SDSS, Filter_SDSS, lightcurve_length_before, lightcurve_length_after)

In [ ]:
print(np.mean(m_JMC[0][2]))

In [ ]:
def GP_normalization(y, y_err):
    return (y - np.mean(y)) / (np.max(y) - np.min(y)), y_err / (np.max(y) - np.min(y)), np.mean(y), np.max(y) - np.min(y)

In [ ]:
def GP_interpolate(t, m, m_err, Type, Filter, Filter_wavelength, filename1, color):
    
    data = []
    padding_point = []
    
    data_mean = []
    data_range = []
    
    m1 = []
    m1_err = []
    
    selected = [180]
    
    #for i in selected:
    for i in range(len(t)):
        Filter_num = [] # List containing Filter_wavelength for GP fitting, with number of element matching the number of element in x (time)
        x = [] # List of t containing all bands
        y = [] # List of m containing all bands
        y_err = [] # List of m_err containing all bands
        residual = [] # List of GP prediction - actual observation
        a = 'empty' # Store the band number of the longest band timewise
        b = 0 # Store the time duration of the longest band timewise
        
        m1.append([])
        m1_err.append([])
        for j in range(len(Filter)):
            m1[i].append([])
            m1_err[i].append([])
        
        for j in range(len(Filter)):
            # Finding which band has the longest light curve
            if len(t[i][j]) != 0:
                if (t[i][j][-1] - t[i][j][0]) > b:
                    b = t[i][j][-1] - t[i][j][0]
                    a = j

            # Saving the t and m and m_err for all bands for GP fitting
            for k in range(len(t[i][j])):
                x.append(t[i][j][k])
                y.append(m[i][j][k])
                y_err.append(m_err[i][j][k])
                Filter_num.append(Filter_wavelength[j])

        x1 = np.vstack([x, Filter_num]).T # 2D list with both the time and Filter_wavelength used for initial GP fitting


        Filter_pred = [] # List containing Filter_wavelength for GP fitting, with number of element matching the number of element in x (time) 
        x_pred = [] # List of t containing all bands
        for j in range(len(Filter)):
            x_pred_temp = np.linspace(t[i][a][0], t[i][a][-1], 500)
            for k in range(500):
                x_pred.append(x_pred_temp[k])
                Filter_pred.append(Filter_wavelength[j])


        x_pred1 = np.vstack([x_pred, Filter_pred]).T # 2D list with both the artificial time list and Filter_wavelength used for GP generation
        
        
        # Normalization
        y, y_err, y_mean, y_range = GP_normalization(y, y_err) # Normalizing the magnitude and magnitude error such that the GP mean is 0
        
        for j in range(len(Filter)):
            m1[i][j] = (m[i][j] - y_mean) / y_range
            m1_err[i][j] = m_err[i][j] / y_range
        
        data_mean.append(y_mean)
        data_range.append(y_range)
        
        
        
        kernel = np.var(y)*kernels.Matern32Kernel(metric=[100,1], ndim=2) + kernels.ConstantKernel(log_constant=0, ndim=2) # 3/2 Matern Kernel and Constant Kernel
        gp = george.GP(kernel)
        
        gp.compute(x1, y_err) # Initial GP fitting

        def neg_ln_like(p):
            gp.set_parameter_vector(p)
            return -gp.log_likelihood(np.array(y))

        def grad_neg_ln_like(p):
            gp.set_parameter_vector(p)
            return -gp.grad_log_likelihood(np.array(y))

        try:
            result = minimize(neg_ln_like, gp.get_parameter_vector(), jac=grad_neg_ln_like, method='L-BFGS-B') # Fitting for finding the optimum parameters of the kernel
            print(i, result)
            print(i, 'converged successfully')
            gp.recompute()
            pred, pred_var = gp.predict(y, x_pred1, return_var=True) # GP generation with artifical time
        except:
            print(i, 'failed to converge')
            continue


        # Plot out the result of the GP generation with artificial time
        fig, axs = plt.subplots(1, 2, figsize=(35, 9))
        #plt.gca().invert_yaxis()
        for j in range(len(Filter)):
            for k in range(2):
                axs[k].plot(x_pred[j*500:(j+1)*500], pred[j*500:(j+1)*500], color=color[j], lw=1.5, alpha=0.8, label=Filter[j])
                axs[k].fill_between(x_pred[j*500:(j+1)*500], pred[j*500:(j+1)*500] - np.sqrt(pred_var[j*500:(j+1)*500]), pred[j*500:(j+1)*500] + np.sqrt(pred_var[j*500:(j+1)*500]),
                                color=color[j], alpha=0.2)
                axs[k].errorbar(t[i][j], m1[i][j], yerr=m1_err[i][j], fmt='.', color=color[j], capsize=0)
                axs[0].set_xlim(t[i][a][0], t[i][a][-1])
                axs[1].set_xlim(-50, 135)
                axs[1].set_ylim(min(pred[j*500:(j+1)*500])-0.5, max(pred[j*500:(j+1)*500])+0.5)
        
        for k in range(2):
            axs[k].invert_yaxis()
            axs[k].grid()
            axs[k].legend()
            #axs[k].set_title(filename[filename1[i]], color='white')
            axs[k].set_title(filename[filename1[i]]+' '+Type[i], color='white')
            
        plt.show()


        pred_residual, pred_var_residual = gp.predict(y, x1, return_var=True) # GP generation with actual observation
        #print('shape of pred_residual', pred_residual.shape)


        fig, axs = plt.subplots(1, len(Filter)+1, figsize=(35, 6))
        l = 0

        # Calculate the residual
        for j in range(len(Filter)+1):
            residual.append([])
            if j != len(Filter): # Residual calculation for each single band
                for k in range(len(m1[i][j])):
                    residual[j].append(-1*(m1[i][j][k] - pred_residual[l+k])) # Notice the sign is flipped for easier observation for our ape brain
                l += len(m1[i][j]) # id calculation as pred_residual is just a 1D long list containing all bands
            else:
                for k in range(len(y)): # Residual calculation for all bands
                    residual[j].append(-1*(y[k] - pred_residual[k])) # Again the sign is flipped

            mu, sigma = norm.fit(residual[j]) # Fit a Gaussian distribution on the residual

            # Plotting the residual
            values,bins,_ = axs[j].hist(residual[j], bins=20, color=color[j], density=False, alpha=0.7)
            area = sum(np.diff(bins)*values)

            if len(residual[j]) == 0: # Skip empty bands
                continue

            gcxmin = np.min(residual[j])
            gcxmax = np.max(residual[j])
            gcx = np.linspace(gcxmin, gcxmax, 100)
            gc = area*norm.pdf(gcx, mu, sigma)

            axs[j].plot(gcx, gc, color=color[j], linestyle='--', linewidth=2)
            axs[j].axvline(x=0, color='k', label='0')
            axs[j].axvline(x=mu, color=color[j], linestyle='--', label='mean')
            axs[j].legend()
            axs[j].grid()
            axs[j].set_title('mean = '+'%.3f'%mu+', var = '+'%.3f'%sigma, color='white')

        plt.show()


        # Temporary variable per SN
        data_t = [] # Store the time from GP interpolation
        data_m = [] # Store the magnitude from GP interpolation
        data_m_err = [] # Store the error of magnitude from GP interpolation
        data_temp = [] # Store all temporary data per SN, this list will be saved to the actual data[] within every loop

        Filter_pred = []
        data_t_temp = np.linspace(t[i][a][0], t[i][a][-1], int(t[i][a][-1] - t[i][a][0])+1) # Time list for GP interpolation, data points with 1 day apart
        padding_point.append(len(data_t_temp)) # Padding was used after the length of the GP interpolation
        
        # Initialize time list for GP interpolation
        for j in range(len(Filter)):   
            for k in range(len(data_t_temp)):
                data_t.append(data_t_temp[k])
                Filter_pred.append(Filter_wavelength[j])

        data_t1 = np.vstack([data_t, Filter_pred]).T
        data_m_temp, data_m_err_temp = gp.predict(y, data_t1, return_var=True)

        lightcurve_length = np.arange(lightcurve_length_before, lightcurve_length_after, 1) # The whole length of time list
        
        # LC padding
        for j in range(len(Filter)):
            data_m.append([])
            data_m_err.append([])
            for k in lightcurve_length:
                if k < t[i][a][0]:
                    pass
                elif k > t[i][a][-1]:
                    #print((j+1)*(len(data_t_temp))-1)
                    data_m[j].append(data_m_temp[(j+1)*(len(data_t_temp))-1])
                    data_m_err[j].append(np.sqrt(data_m_err_temp[(j+1)*(len(data_t_temp))-1]))
                else:
                    #print(j*(len(data_t_temp))+k-int(t[i][a][0]))
                    data_m[j].append(data_m_temp[j*(len(data_t_temp))+k-int(t[i][a][0])])
                    data_m_err[j].append(np.sqrt(data_m_err_temp[j*(len(data_t_temp))+k-int(t[i][a][0])]))
            for k in range(len(lightcurve_length)-len(data_m[j])): # This part was used if the length of data is less than 185
                data_m[j].append(data_m[j][-1])
                data_m_err[j].append(np.sqrt(data_m_err[j][-1]))

        lightcurve_length = lightcurve_length + (t[i][a][0] - lightcurve_length_before) # Time list

        #print('data_m[0]=',np.array(data_m[0]).shape)

        data_temp.append(lightcurve_length)
        for j in range(len(Filter)):
            data_temp.append(data_m[j])
        for j in range(len(Filter)):
            data_temp.append(data_m_err[j])
        data.append([data_temp[j] for j in range(len(data_temp))])
        
        # Plot out the data input
        plt.figure(figsize=(10, 6))
        plt.gca().invert_yaxis()
        plt.grid()
        for j in range(len(Filter)):
            #plt.scatter(lightcurve_length, data_m[j], color=color[j], s=0.3)
            plt.errorbar(lightcurve_length, data_m[j], yerr=data_m_err[j], fmt='.', color=color[j], lw=0.8)
        plt.show()

    data = np.array(data, dtype=object)
    print('data shape=', data.shape)
    
    return data, data_mean, data_range

In [ ]:
Filter_wavelength_JMC = [3.663, 4.361, 5.448, 6.407, 7.980]
color = ['darkviolet', 'royalblue', 'seagreen', 'crimson', 'maroon', 'k']
data_JMC, mean_JMC, range_JMC = GP_interpolate(t_JMC, m_JMC, m_err_JMC, Type_JMC, Filter_JMC, Filter_wavelength_JMC, filename_JMC, color)

In [ ]:
Filter_wavelength_SDSS_p = [3.542, 4.724, 6.202, 7.759] # From Spanish Virtual Observatory
color = ['darkviolet', 'seagreen', 'crimson', 'maroon', 'k']
data_SDSS_p, mean_SDSS_p, range_SDSS_p = GP_interpolate(t_SDSS_p, m_SDSS_p, m_err_SDSS_p, Type_SDSS_p, Filter_SDSS_p, Filter_wavelength_SDSS_p, filename_SDSS_p, color)

In [ ]:
Filter_wavelength_SDSS = [3.608, 4.671, 6.141, 7.457] # From Spanish Virtual Observatory
color = ['darkviolet', 'seagreen', 'crimson', 'maroon', 'k']
data_SDSS, mean_SDSS, range_SDSS = GP_interpolate(t_SDSS, m_SDSS, m_err_SDSS, Type_SDSS, Filter_SDSS, Filter_wavelength_SDSS, filename_SDSS, color)

In [ ]:
'''Conversion from SDSS_p to SDSS

Equations from Andrew Pickles
Las Cumbres Observatory Global Telescope, Santa Barbara, CA
The 2010 STScI Calibration Workshop
'''

def unnormalization(data, mean, rang):
    for i in range(data.shape[0]):
        data[i,:,:] = data[i,:,:]*rang[i] + mean[i]
    return data

data_SDSS_p = unnormalization(data_SDSS_p, mean_SDSS_p, range_SDSS_p)
data_JMC = unnormalization(data_JMC, mean_JMC, range_JMC)

print(data_SDSS_p[0,1,:])

def renormalization(i, data):
    y = []
    
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            y.append(data[i,j,k])
    
    y_mean = np.mean(y)
    y_range = np.max(y) - np.min(y)
    
    return data[i,:,:]
    
    

data_SDSS_temp = np.zeros((data_SDSS_p.shape))

for i in range(data_SDSS_p.shape[0]):
    
    data_SDSS_temp[i,0,:] =         data_SDSS_p[i,0,:]
    data_SDSS_temp[i,1,:] = 0.037 + data_SDSS_p[i,1,:] + 0.060 * (data_SDSS_p[i,1,:] - data_SDSS_p[i,2,:] - 0.53)
    data_SDSS_temp[i,2,:] = 0.010 + data_SDSS_p[i,2,:] + 0.035 * (data_SDSS_p[i,2,:] - data_SDSS_p[i,3,:] - 0.21)
    data_SDSS_temp[i,3,:] =       + data_SDSS_p[i,3,:] + 0.041 * (data_SDSS_p[i,2,:] - data_SDSS_p[i,3,:] - 0.21)
    
    
    y = []
    
    for j in range(data_SDSS_temp.shape[1]):
        for k in range(data_SDSS_temp.shape[2]):
            y.append(data_SDSS_temp[i,j,k])
    
    y_mean = np.mean(y)
    y_range = np
    